In [15]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential
from pipeline_utils import (
    COMPUTE, CONCURRENT_TRIALS, DELAY_EVALUATION, EVALUATION_INTERVAL, GOAL, METRIC, SAMPLING_ALGORITHM, TIMEOUT_PLUS, TOTAL_TRIALS, #SWEEP
    SUBSCRIPTION, RESOURCE_GROUP, WS_NAME,  #AUTHENTICATE
    MULT_NBC_PATH,    #COMPONENTS PATHS
    GINI_PATH,    #FEAT SEL PATHS
    PREP_DATA_PATH,  #PREP DATA PATH
    MULT_NBC_BY_GINI,    #PIPELINES
    GINI,   #FEAT SEL
    get_experiment_names,   #FUNCTIONS
)

# authenticate
credential = DefaultAzureCredential()

# Get a handle to the workspace
ml_client = MLClient(
    credential=credential,
    subscription_id=SUBSCRIPTION,
    resource_group_name=RESOURCE_GROUP,
    workspace_name=WS_NAME,
)

In [16]:
TRAIN_DATAS = [
    "vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017.csv"
]

TEST_DATAS = [
    "vrex_encoded_tf_idf_2018_2019_2020_2021.csv"
]

version = "v1"

arr_data_to_train = []
arr_data_to_test = []

for to_train, to_test in zip(TRAIN_DATAS, TEST_DATAS):
    data_to_train = ml_client.data.get(name=to_train.split(".")[0], version=version)
    arr_data_to_train.append(data_to_train)
    print(f"Data to train asset URI: {data_to_train.path} - name: {to_train.split('.')[0]}")

    data_to_test = ml_client.data.get(name=to_test.split('.')[0], version=version)
    arr_data_to_test.append(data_to_test)
    print(f"Data to test asset URI: {data_to_test.path} - name: {to_test.split('.')[0]}")

Data to train asset URI: azureml://subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests/datastores/workspaceblobstore/paths/LocalUpload/3cfdebd258ea2db7363c55bb841ca887/vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017.csv - name: vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017
Data to test asset URI: azureml://subscriptions/1f51a68b-69d8-4818-bf63-7c5e10b81967/resourcegroups/luiz.victor.dev-rg/workspaces/tcc-tests/datastores/workspaceblobstore/paths/LocalUpload/3992df39ce41e34367e629d6656f4e9f/vrex_encoded_tf_idf_2018_2019_2020_2021.csv - name: vrex_encoded_tf_idf_2018_2019_2020_2021


In [17]:
# importing the Component Package
from azure.ai.ml import load_component

data_prep_component = load_component(source=PREP_DATA_PATH)
data_prep_component = ml_client.create_or_update(data_prep_component)

feat_sel_component = load_component(source=GINI_PATH)
feat_sel_component = ml_client.create_or_update(feat_sel_component)

mult_nbc_train = load_component(source=MULT_NBC_PATH)
mult_nbc_train = ml_client.create_or_update(mult_nbc_train)


In [18]:

# the dsl decorator tells the sdk that we are defining an Azure Machine Learning pipeline
from azure.ai.ml import dsl, Input, Output
from azure.ai.ml.sweep import Choice, Uniform, MedianStoppingPolicy

In [19]:
@dsl.pipeline(
    name=MULT_NBC_BY_GINI,
    compute=COMPUTE,
    description="E2E data_perp-train pipeline",
)
def train_mult_nbc_pipeline(
    data_to_train,
    data_to_test,
    feature_quantity,
    flag_remove_null_values,
    flag_remove_values_by_percentage,
    percentage_to_remove_column,
):

    data_prep_job = data_prep_component(
        data_to_train=data_to_train,
        data_to_test=data_to_test,
        flag_remove_null_values=flag_remove_null_values,
        flag_remove_values_by_percentage=flag_remove_values_by_percentage,
        percentage_to_remove_column=percentage_to_remove_column,
    )

    feat_sel_job = feat_sel_component(
        train_data=data_prep_job.outputs.train_data,
        test_data=data_prep_job.outputs.test_data,
        feature_quantity=feature_quantity,
    )

    mult_nbc_train_job = mult_nbc_train(
        train_data=feat_sel_job.outputs.train_data_feat_sel,  
        test_data=feat_sel_job.outputs.test_data_feat_sel,  
        alpha=Choice(values=[0.01, 0.1, 0.5, 1.0, 2, 5]),
        fit_prior=Choice(values=[True, False]),
    )

    sweep_step_to_mult_nbc = mult_nbc_train_job.sweep(
        compute=COMPUTE,
        sampling_algorithm=SAMPLING_ALGORITHM,
        primary_metric=METRIC,
        goal=GOAL,
    )

    sweep_step_to_mult_nbc.set_limits(max_total_trials=2*TOTAL_TRIALS, max_concurrent_trials=CONCURRENT_TRIALS, timeout=TIMEOUT_PLUS)

    sweep_step_to_mult_nbc.early_termination = MedianStoppingPolicy(delay_evaluation=DELAY_EVALUATION, evaluation_interval=EVALUATION_INTERVAL)


In [20]:
pipelines = []
n_features = [100, 50]

for n_feature in n_features:
    for data_to_train, data_to_test in zip(arr_data_to_train, arr_data_to_test):

        pipelines.append(train_mult_nbc_pipeline(
            data_to_train=Input(type="uri_file", path=data_to_train.path),
            data_to_test=Input(type="uri_file", path=data_to_test.path),
            feature_quantity=n_feature,
            flag_remove_null_values=False,
            flag_remove_values_by_percentage=False,
            percentage_to_remove_column=0,
        ))


In [21]:
experiment_names = get_experiment_names(TRAIN_DATAS, TEST_DATAS, GINI, n_features)
for pipeline, experiment_name in zip(pipelines, experiment_names):
    pipeline_job = ml_client.jobs.create_or_update(
        pipeline,
        experiment_name=experiment_name,
    )
    ml_client.jobs.stream(pipeline_job.name)

vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_GaussianNB_2024_06_20_21_07_51
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_GradientBoostingClassifier_2024_06_20_21_07_51
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_RandomForestClassifier_2024_06_20_21_07_51
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_SVC_2024_06_20_21_07_51
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_XGBoost_2024_06_20_21_07_51
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016_2017_tested_vrex_encoded_tf_idf_2018_2019_2020_2021_Gini_GaussianNB_2024_06_20_21_07_51
vrex_encoded_tf_idf_2008_2009_2010_2011_2012_2013_2014_2015_2016

Bad pipe message: %s [b'\xf8F\x0b\xc0\xe3\xe1\xc1\x96\xf0\x03\xc1\x87\xcb\xe6\x90n\xf0\x86 \x05\x93\x06m\x19h\x02}\xcc\xd2\xc4d\xb4%\x1b\x18\xaai\x1d\xa9\xb1\xbc)g(=\xf9\xf2\xe2&\x1eE\x00\x08\x13\x02\x13\x03\x13\x01\x00\xff\x01\x00\x00\x8f\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00']
Bad pipe message: %s [b'\x1d\x00\x17\x00\x1e\x00\x19\x00\x18']
Bad pipe message: %s [b'j\xf3\x18\x18\x0c#Q\xc6ZO\xe22#D\xech\xee0\x00\x00|\xc0,\xc00\x00\xa3\x00\x9f\xcc\xa9\xcc\xa8\xcc\xaa\xc0\xaf\xc0\xad\xc0\xa3\xc0\x9f\xc0]\xc0a\xc0W', b"\xc0+\xc0/\x00\xa2\x00\x9e\xc0\xae\xc0\xac\xc0\xa2\xc0\x9e\xc0\\\xc0`\xc0V\xc0R\xc0$\xc0(\x00k\x00j\xc0#\xc0'\x00g\x00@\xc0\n\xc0\x14\x009\x008\xc0\t\xc0\x13\x003\x002\x00\x9d\xc0\xa1\xc0\x9d\xc0Q\x00\x9c\xc0\xa0\xc0\x9c\xc0P\x00=\x00<\x005\x00/\x00\x9a\x00\x99\xc0\x07\xc0\x11\x00\x96\x00\x05\x00\xff\x01\x00\x00j\x00\x00\x00\x0e\x00\x0c\x00\x00\t127.0.0.1\x00\x0b\x00\x04\x03\x00\x01\x02\x00\n\x00\x0c\x00\n\x00\x1d\x00\x17